<a href="https://colab.research.google.com/github/Sergey-Kiselev-dev/NN_01_Keras/blob/main/NN_01_03a_GradDesc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Градиентный спуск
===

Сегодня обсудим:

Для чего нужен градиентный спуск для нейронных сетей
1. Что такое градиент
2. Что такое градиентный спуск

Для чего нужен?
---
Чтобы ответить на данный вопрос, давайте возьмем задачу построения и обучения сети для умножения на 3.

Создадим обучающие данные

In [ ]:
import numpy as np

X = np.array([[1], [3], [2], [10], [4], [7], [8]])
y = np.array([[3, 9, 6, 30, 12, 21, 24]]).T

Создадим сеть, она очень простая, состоит из одного слоя и одного нейрона.

input --> 0 --> outpot

In [ ]:
from keras.layers import Dense
from keras.models import Sequential
import tensorflow as tf
tf.random.set_seed(0)

model = Sequential([
    Dense(1, input_shape=(1,), activation='linear')
])

model.summary()

Весов у нас выходит две штуки, это вышло из-за того, что для каждого нейрона в линейном слое есть отклонение (bias).

In [ ]:
w1, w0 = model.get_weights()
w1 = w1[0][0]
w0 = w0[0]

w1, w0

Теперь сделаем предсказание этой моделью на одном объекте.

In [ ]:
X[:1]

Предсказание получается очень далекими от истины, потому что сеть еще не знает, для чего её создали.

In [ ]:
model.predict(X[:1])

In [ ]:
w1 * X[:1] + w0

In [ ]:
from keras.activations import linear
linear(w1 * X[:1] + w0)

Оптимизируем/уменьшаем ошибку MSE - а это функция, которая меняется от весов в нейроне.

Можем взять по 100 разных значений весов и посчитать в них MSE и отобразить на трехмерном графике.

Так же отобразим и веса, которые есть на момент инициализации сети.

In [ ]:
from mpl_toolkits.mplot3d.axes3d import Axes3D
import matplotlib.pyplot as plt


def mse(w1, w0):
    y_pred = w1 * X[:, 0] + w0
    return np.mean((y - y_pred) ** 2)


coefs_w1 = np.linspace(-5, 10, num=100)
coefs_w0 = np.linspace(-5, 5, num=100)
w1s, w0s = np.meshgrid(coefs_w1, coefs_w0)


fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')

zs = np.array([mse(i, j) for i, j in zip(np.ravel(w1s), np.ravel(w0s))])
Z = zs.reshape(w1s.shape)

ax.plot_surface(w1s, w0s, Z, alpha=.5)
ax.scatter(w1, w0, mse(w1, w0), c='r', s=5)

ax.set_xlabel(r'$w_1$')
ax.set_ylabel(r'$w_0$')
ax.set_zlabel('MSE')

plt.show()

In [ ]:
coefs_w1 = np.linspace(-5, 10, num=100)
coefs_w0 = np.linspace(-5, 5, num=100)
w1s, w0s = np.meshgrid(coefs_w1, coefs_w0)

zs = np.array([round(mse(i, j)) for i, j in zip(np.ravel(w1s), np.ravel(w0s))])
Z = zs.reshape(w1s.shape)

fig = plt.figure(figsize = (10,7))
plt.imshow(Z, extent=[-5,10, -5,5], origin = 'lower', cmap = 'jet', alpha = 1)
plt.colorbar()
plt.scatter(w1, w0, c='r', s=15, label='start weights')


plt.xlabel('w1', fontsize=11)
plt.ylabel('w0', fontsize=11)

plt.legend(loc="upper right");

Чтобы оказаться в самой минимальной ошибке нужно до нее дойти, а идти до нее можем с помощью градиентного спуска.

Градиентный спуск
---

> **Градиентный спуск** — метод нахождения локального минимума или максимума функции с помощью движения вдоль градиента.

_Градиентом_ функции $f$ называется $n$-мерный вектор из частных производных.

$$ \nabla f(x_{1},...,x_{d}) = \left(\frac{\partial f}{\partial x_{i}}\right)^{d}_{i=1}.$$

К примеру, если функция зависит от трех переменных: $F(x, y, z)$, то её градиент будет равен

$$\nabla f(x, y, z) = (\frac{\partial f}{\partial x}, \frac{\partial f}{\partial y}, \frac{\partial f}{\partial z}) $$

При этом, __градиент задает направление наискорейшего роста функции__. Значит, антиградиент будет показывать направление ее скорейшего убывания, что будет полезно нам в нашей задаче минимизации функционала ошибки.

Давайте попробуем реализовать программно градиентный спуск на нашем примере, чтобы лучше понять как он работает.

Функция, которую здесь оптимизируем - это MSE, её график для конкретно нашей задачи рисовали выше.

#### Вручную

Реализуем две функции:
1. mserror - функция среднеквадратичной ошибки $MSE = \frac{1}{n}\sum_{i=0}^n{(\text{y}_i-\text{y_pred}_i})^2 = \frac{1}{n}\sum_{i=0}^n{(\text{y}_i-(w_1\cdot X_i + w_0)})^2 = \frac{1}{n}\sum_{i=0}^n{(\text{y}_i-w_1\cdot X_i - w_0})^2$


2. gr_mserror - градиент функции MSE. Распишем его отдельно для весов:


$w_1$:
$\frac{∂ MSE}{∂ w_1} = \frac{1 \cdot 2}{n}\sum({y_i -\text{y_pred}_i})\cdot -X$

$w_0$:
$\frac{∂ MSE}{∂ w_0} = \frac{1 \cdot 2}{n}\sum({y_i -\text{y_pred}_i})\cdot -1$

In [ ]:
# функция, определяющая среднеквадратичную ошибку
def mserror(X, w1, w0, y):
    y_pred = w1 * X[:, 0] + w0
    return np.sum((y - y_pred) ** 2) / len(y_pred)

# функция градиента
def gr_mserror(X, w1, w0, y):
    y_pred = w1 * X + w0
    return {'grad_w1': 2/len(y)*np.sum((y - y_pred) * (-X)),
            'grad_w0': 2/len(y)*np.sum((y - y_pred)) * (-1)}

Перед тем, как считать градиенты и перед тем, как запускать градиентный спуск, давайте посчитаем ошибку

In [ ]:
preds = X * w1 + w0
preds

In [ ]:
import pandas as pd

df = pd.DataFrame({
   'true': np.squeeze(y),
   'pred': np.squeeze(preds)
})

df

In [ ]:
np.mean((df['true'] - df['pred']) ** 2)

Инициализация начальной точки

In [ ]:
weights_1 = [w1]
weights_0 = [w0]
grad = gr_mserror(X, w1, w0, y)
grad

In [ ]:
next_w_1 = w1 - grad['grad_w1']
next_w_0 = w0 - grad['grad_w0']

next_w_1, next_w_0

Получились очень большие веса, давайте посчитаем на них ошибку

In [ ]:
preds = X * next_w_1 + next_w_0

df = pd.DataFrame({
   'true': np.squeeze(y),
   'pred': np.squeeze(preds)
})

df

In [ ]:
np.mean((df['true'] - df['pred']) ** 2)

Совсем гигантская получилась ошибка, значит что-то пошло не так. А именно пошли не так градиенты, они очень большие, то есть функция растет быстрее, чем она уменьшается в другом направлении.

Чтобы этого избежать можем использовать скорость обучения (learning rate).

> **Скорость обучения** — это параметр градиентных алгоритмов, позволяющий управлять величиной коррекции весов на каждой итерации.

In [ ]:
grad = gr_mserror(X, w1, w0, y)
grad

In [ ]:
lr = 0.01

next_w_1 = w1 - lr * grad['grad_w1']
next_w_0 = w0 - lr * grad['grad_w0']

weights_1.append(next_w_1)
weights_0.append(next_w_0)

next_w_1, next_w_0

Получились веса не такие большие, как после первого запуска.

Посчитаем ошибку

In [ ]:
preds = X * next_w_1 + next_w_0

df = pd.DataFrame({
   'true': np.squeeze(y),
   'pred': np.squeeze(preds)
})

df

In [ ]:
np.mean((df['true'] - df['pred']) ** 2)

Движемся в правильном направлении, судя по ошибке.

In [ ]:
coefs_w1 = np.linspace(-5, 10, num=100)
coefs_w0 = np.linspace(-5, 5, num=100)
w1s, w0s = np.meshgrid(coefs_w1, coefs_w0)

zs = np.array([round(mse(i, j)) for i, j in zip(np.ravel(w1s), np.ravel(w0s))])
Z = zs.reshape(w1s.shape)

fig = plt.figure(figsize = (10,7))
plt.imshow(Z, extent=[-5,10, -5,5], origin = 'lower', cmap = 'jet', alpha = 1)
plt.colorbar()


plt.plot(weights_1, weights_0, label='gradient descent', c='r')
plt.scatter(weights_1, weights_0, marker='*', c='r')

plt.xlabel('w1', fontsize=11)
plt.ylabel('w0', fontsize=11)

plt.legend(loc="upper right");

In [ ]:
grad = gr_mserror(X, next_w_1, next_w_0, y)
grad

In [ ]:
next_w_1 = next_w_1 - lr * grad['grad_w1']
next_w_0 = next_w_0 - lr * grad['grad_w0']

weights_1.append(next_w_1)
weights_0.append(next_w_0)

next_w_1, next_w_0

In [ ]:
preds = X * next_w_1 + next_w_0

df = pd.DataFrame({
   'true': np.squeeze(y),
   'pred': np.squeeze(preds)
})

np.mean((df['true'] - df['pred']) ** 2)

Движемся в правильном направлении, судя по ошибке.

In [ ]:
coefs_w1 = np.linspace(-5, 10, num=100)
coefs_w0 = np.linspace(-5, 5, num=100)
w1s, w0s = np.meshgrid(coefs_w1, coefs_w0)

zs = np.array([round(mse(i, j)) for i, j in zip(np.ravel(w1s), np.ravel(w0s))])
Z = zs.reshape(w1s.shape)

fig = plt.figure(figsize = (10,7))
plt.imshow(Z, extent=[-5,10, -5,5], origin = 'lower', cmap = 'jet', alpha = 1)
plt.colorbar()


plt.plot(weights_1, weights_0, label='gradient descent', c='r')
plt.scatter(weights_1, weights_0, marker='*', c='r')

plt.xlabel('w1', fontsize=11)
plt.ylabel('w0', fontsize=11)

plt.legend(loc="upper right");

In [ ]:
# количество итерация
n = 100

for i in range(n):
    cur_weight_1 = next_w_1
    cur_weight_0 = next_w_0
    grad = gr_mserror(X, cur_weight_1, cur_weight_0, y)

    next_w_1 = cur_weight_1 - lr * grad['grad_w1']
    next_w_0 = cur_weight_0 - lr * grad['grad_w0']

    weights_1.append(next_w_1)
    weights_0.append(next_w_0)

preds = X * cur_weight_1 + cur_weight_0

df = pd.DataFrame({
   'true': np.squeeze(y),
   'pred': np.squeeze(preds)
})

df

In [ ]:
np.mean((df['true'] - df['pred']) ** 2)

In [ ]:
coefs_w1 = np.linspace(-5, 10, num=100)
coefs_w0 = np.linspace(-5, 5, num=100)
w1s, w0s = np.meshgrid(coefs_w1, coefs_w0)

zs = np.array([round(mse(i, j)) for i, j in zip(np.ravel(w1s), np.ravel(w0s))])
Z = zs.reshape(w1s.shape)

fig = plt.figure(figsize = (10,7))
plt.imshow(Z, extent=[-5,10, -5,5], origin = 'lower', cmap = 'jet', alpha = 1)
plt.colorbar()


plt.plot(weights_1, weights_0, label='gradient descent', c='r')
plt.scatter(weights_1, weights_0, marker='*', c='r')

plt.xlabel('w1', fontsize=11)
plt.ylabel('w0', fontsize=11)

plt.legend(loc="upper right");

#### Алгоритм градиентного спуска

1. Инициализация начальной точки
2. Цикл по k = 1,2,3,...:

- $ w_{k} = w_{k-1} - \eta\nabla f(w_{k-1}) $


#### Через Keras

In [ ]:
df = pd.DataFrame({
   'true': np.squeeze(y),
   'pred': np.squeeze(model.predict(X))
})

np.mean((df['true'] - df['pred']) ** 2)

In [ ]:
model.compile(optimizer='sgd', loss='mse', metrics=[tf.keras.metrics.CategoricalAccuracy()])

model.get_weights()[0][0][0], model.get_weights()[1][0]

In [ ]:
n_epochs = 10
weights = [[model.get_weights()[0][0][0], model.get_weights()[1][0]]]

for i in range(n_epochs):
    model.fit(X, y)
    weights.append([model.get_weights()[0][0][0], model.get_weights()[1][0]])

weights = np.array(weights)
weights

In [ ]:
import pandas as pd

df = pd.DataFrame({
   'true': np.squeeze(y),
   'pred': np.squeeze(model.predict(X))
})

df

In [ ]:
np.mean((df['true'] - df['pred']) ** 2)

In [ ]:
coefs_w1 = np.linspace(-5, 10, num=100)
coefs_w0 = np.linspace(-5, 5, num=100)
w1s, w0s = np.meshgrid(coefs_w1, coefs_w0)


fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')

zs = np.array([mse(i, j) for i, j in zip(np.ravel(w1s), np.ravel(w0s))])
Z = zs.reshape(w1s.shape)

ax.plot_surface(w1s, w0s, Z, alpha=.5)

mses = []
for weight1, weight0 in weights:
    mses.append(mse(weight1, weight0))

ax.plot(weights[:, 0], weights[:, 1], mses, label='gradient descent', c='r')
ax.scatter(weights[:, 0], weights[:, 1], mses, c='r', marker='*', s=50)


ax.set_xlabel(r'$w_1$')
ax.set_ylabel(r'$w_0$')
ax.set_zlabel('MSE')

plt.show()

In [ ]:
coefs_w1 = np.linspace(-5, 10, num=100)
coefs_w0 = np.linspace(-5, 5, num=100)
w1s, w0s = np.meshgrid(coefs_w1, coefs_w0)

zs = np.array([round(mse(i, j)) for i, j in zip(np.ravel(w1s), np.ravel(w0s))])
Z = zs.reshape(w1s.shape)

fig = plt.figure(figsize = (10,7))
plt.imshow(Z, extent=[-5,10, -5,5], origin = 'lower', cmap = 'jet', alpha = 1)
plt.colorbar()


plt.plot(weights[:, 0], weights[:, 1], label='gradient descent', c='r')
plt.scatter(weights[:, 0], weights[:, 1], marker='*', c='r')

plt.xlabel('w1', fontsize=11)
plt.ylabel('w0', fontsize=11)

plt.legend(loc="upper right");